<a href="https://colab.research.google.com/github/roque-alfaro/taller-eiv-2026/blob/main/3_Ejercicio_Cubo_multidimensional_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio: creación de un cubo
En este ejercicio vamos a armar una tabla de datos multidimensional para el análisis de las brechas de consultas de especialidad médica (CNE). Para ello haremos lo siguiente:

1. Generar un modelo estrella con las métricas y dimensiones de análisis

2. Generar tablas fact y dimensiones

3. Exportar a excel

 # Configurar área de trabajo

In [1]:
# importar librerías
import pandas as pd
import os

# definir carpetas de trabajo
datos_normalizados = 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_normalizados/'
datos_cubos = '/content/cubos/'
if not os.path.exists(datos_cubos): os.makedirs(datos_cubos)

# cargar datos
centros = pd.read_parquet(datos_normalizados + 'Centros.parquet')
especialidades = pd.read_parquet(datos_normalizados + 'Especialidades.parquet')
demanda = pd.read_parquet(datos_normalizados + 'Demanda.parquet')
produccion = pd.read_parquet(datos_normalizados + 'Produccion.parquet')

# Crear tabla FACT
Construiremos una tabla con los campos:

- Centro_cod: código del centro de salud
- Especialidad_cod: código de la especialidad médica (prestación)
- Mes: mes del año 2021
- CNE_solicitadas: solicitudes de interconsultas recibidas por establecimiento, especialidad y mes
- CNE_producidas: consultas nuevas de especialidad producidas por establecimientos, especialidad y mes



In [2]:
# Examinemos los campos que tienen las tablas de demanda y producción
# Con el cruce de estas tablas podemos obtener las métricas fundamentales para el análisis
print(demanda.head(3), end='\n\n')
print(produccion.head(3))

  Especialidad  Centro_destino   Año  Mes  Interconsultas
0       07-053          114101  2021    1             822
1       07-053          114101  2021    3             748
2       07-044          114101  2021    1             673

    Año  Mes  Centro  Nuevas  Controles  Codigo
0  2021    1  114101      33        192  07-047
1  2021    1  114101     239        325  07-030
2  2021    1  114101       8         82  07-039


In [3]:
# Construimos una función para generar la tabla fact

def generar_tabla_fact():

  # Obtener datos de demanda y producción de consultas nuevas
  dda = demanda[['Centro_destino','Especialidad','Mes','Interconsultas']].copy()
  dda = dda.rename(columns={'Centro_destino':'Centro_cod','Especialidad':'Especialidad_cod','Interconsultas':'CNE_solicitadas'})
  prod = produccion[['Centro','Codigo','Mes','Nuevas']].copy()
  prod = prod.rename(columns={'Centro':'Centro_cod','Codigo':'Especialidad_cod','Nuevas':'CNE_producidas'})

  # Cruzar producción y demanda
  fact = dda.merge(prod, how='outer', on = ['Centro_cod','Especialidad_cod','Mes'])

  # Complatemos los campos vacíos con 0 y guardamos las métricas como números enteros
  fact.fillna(0, inplace=True)
  fact[['CNE_solicitadas','CNE_producidas']] = fact[['CNE_solicitadas','CNE_producidas']].astype(int)
  return fact

# Guardamos y revisamos la tabla resultante
fact = generar_tabla_fact()

print(fact.info())
fact

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2080 entries, 0 to 2079
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Centro_cod        2080 non-null   int64 
 1   Especialidad_cod  2080 non-null   object
 2   Mes               2080 non-null   int64 
 3   CNE_solicitadas   2080 non-null   int64 
 4   CNE_producidas    2080 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 81.4+ KB
None


,Centro_cod,Especialidad_cod,Mes,CNE_solicitadas,CNE_producidas
0,114101,07-002,1,2,0
1,114101,07-002,4,0,1
2,114101,07-002,5,0,60
3,114101,07-002,6,0,83
4,114101,07-002,7,0,88
...,...,...,...,...,...
2075,200282,07-061,8,0,37
2076,200282,07-061,9,0,47
2077,200282,07-061,10,0,35
2078,200282,07-061,11,0,40


## 🏁 Discusión
1. ¿Cuáles son las métricas que registra esta tabla?
1. ¿Cuáles son los hechos que registra esta tabla? ¿Cuántos exiten?
2. ¿Cuál es el nivel de detalle con que se describen los hechos (granularidad)?
1. ¿A partir de esta tabla es posible consultar los hechos a nivel diario? ¿Trimestral? ¿Anual?

# Dimensión Periodo
Aunque nuestro análisis sólo considera un año, podríamos querer analizar los datos a nivel trimestral o anual. Construiremos una tabla con los campos

- Mes: llave primaria
- Trimestre
- Año

**⚠** Si trabajáramos con múltiples años necesitaríamos definir otra llave primaria para esta dimensión

In [4]:
# Creamos un dataframe vacío para agregar los campos
dim_periodo = pd.DataFrame()
dim_periodo['Mes'] = range(1,13)
dim_periodo['Trimestre'] = [1,1,1,2,2,2,3,3,3,4,4,4]
dim_periodo['Año'] = 2021
dim_periodo

,Mes,Trimestre,Año
0,1,1,2021
1,2,1,2021
2,3,1,2021
3,4,2,2021
4,5,2,2021
5,6,2,2021
6,7,3,2021
7,8,3,2021
8,9,3,2021
9,10,4,2021


# Dimensión Especialidad
Crearemos una tabla con los siguientes campos:

- Especialidad_cod: código de la especialidad (llave primaria)
- Especialidad: nombre de la especialidad
- Alto_riesgo: especialidad con mayor mortalidad para tiempos de espera extensos según Martínez et al ([2019](https://doi.org/10.1186/s12889-019-6526-6))
- REM: si se incluye en reportes estadísticos mensuales


In [5]:
# Examinemos la tabla especialidades para identificar los campos existentes y los que faltan
especialidades.sample(5)

,Codigo,Nombre,Nombre_prog,Codigo_prog,Codigo_rem
54,07-055,CARDIOLOGÍA PEDIÁTRICA,CARDIOLOGÍA PEDIÁTRICA,7024219.0,7024219.0
6,07-007,CIRUGÍA DE TÓRAX,CIRUGÍA DE TÓRAX,7022700.0,7022700.0
37,07-038,NEFROLOGÍA PEDIÁTRICO,NEFROLOGÍA PEDIÁTRICO,7020901.0,7020901.0
17,07-018,GASTROENTEROLOGÍA ADULTO,GASTROENTEROLOGÍA ADULTO,7020700.0,7020700.0
0,07-001,ANATOMÍA PATOLÓGICA,ANATOMÍA PATOLÓGICA,7030997.0,NaN


In [6]:
# Generemos la dimensión especialidad con una función especial

def crear_dim_especialidades():

  # creamos una copia de especialidades con los campos que necesitamos
  esp = especialidades[['Codigo','Nombre','Codigo_rem']].copy()

  esp.rename(columns={'Codigo':'Especialidad_cod',
                      'Nombre':'Especialidad'},
             inplace = True)

  # Generamos campo REM
  esp['Especialidad_REM'] = 'No'
  esp.loc[esp['Codigo_rem'].notna(), 'Especialidad_REM'] = 'Si'
  del esp['Codigo_rem']

  # Identificamos las especialidades de alto riesgo: CARDIOLOGIA, CIRUGIA CARDIOVASCULAR, GASTROENTEROLOGIA ADULTO, NEFROLOGIA ADULTO, NEUROLOGIA ADULTO, UROLOGIA
  esp['Riesgo_mortalidad'] = 'No'
  alto_riesgo = ['07-003','07-006','07-018','07-037','07-041','07-054']
  esp.loc[esp['Especialidad_cod'].isin(alto_riesgo), 'Riesgo_mortalidad'] = 'Si'

  return esp

dim_especialidad = crear_dim_especialidades()

# Veamos el resultado
print(dim_especialidad.info())
dim_especialidad.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Especialidad_cod   68 non-null     object
 1   Especialidad       68 non-null     object
 2   Especialidad_REM   68 non-null     object
 3   Riesgo_mortalidad  68 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB
None


,Especialidad_cod,Especialidad,Especialidad_REM,Riesgo_mortalidad
62,07-063,NUTRIÓLOGO,Si,No
33,07-034,MEDICINA MATERNO INFANTIL,No,No
18,07-019,GASTROENTEROLOGÍA PEDIÁTRICA,Si,No
56,07-057,CIRUGÍA PLASTICA Y REPARADORA PEDIÁTRICA,Si,No
48,07-049,PSIQUIATRÍA PEDIÁTRICA Y DE LA ADOLESCENCIA,Si,No


## 🏁 Discusión
1. ¿Qué diferencia a esta tabla de la tabla normalizada *Especialidades*?
2. ¿Cuál es el propósito de cada una?

# 🍎 Desafío: Crear Dimensión Establecimiento
Cree un dataframe `dim_centro` con los siguientes campos:

- Centro_cod: código DEIS del centro (llave primaria)
- Centro: nombre del centro de salud
- Complejidad: nivel de complejidad del centro
- Servicio: Servicio de Salud del cual depende

In [11]:
print(centros.columns.tolist())
centros.head(3)


['Codigo', 'Nombre', 'Dependencia', 'Nombre Comuna', 'Nivel de Complejidad']


,Codigo,Nombre,Dependencia,Nombre Comuna,Nivel de Complejidad
0,126704,Hospital Comunitario Cristina Calderón de Puer...,Servicio de Salud Magallanes,Cabo de Hornos,Baja Complejidad
1,126204,Hospital Naval (Puerto Williams),SEREMI De Magallanes y la Antártica Chilena,Cabo de Hornos,Mediana Complejidad
2,126412,Posta de Salud Rural Cameron,Servicio de Salud Magallanes,Timaukel,Baja Complejidad


In [13]:
# Crear dim_centro desde la tabla que tiene:
# ['Codigo','Nombre','Dependencia','Nombre Comuna','Nivel de Complejidad']

dim_centro = centros[["Codigo", "Nombre", "Nivel de Complejidad", "Dependencia"]].copy()

dim_centro = dim_centro.rename(columns={
    "Codigo": "Centro_cod",
    "Nombre": "Centro",
    "Nivel de Complejidad": "Complejidad",
    "Dependencia": "Servicio"
})

# Dejar 1 fila por centro
dim_centro = dim_centro.drop_duplicates(subset=["Centro_cod"]).reset_index(drop=True)

print(dim_centro.info())
dim_centro.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4749 entries, 0 to 4748
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Centro_cod   4749 non-null   int64 
 1   Centro       4749 non-null   object
 2   Complejidad  4610 non-null   object
 3   Servicio     4749 non-null   object
dtypes: int64(1), object(3)
memory usage: 148.5+ KB
None


,Centro_cod,Centro,Complejidad,Servicio
0,126704,Hospital Comunitario Cristina Calderón de Puer...,Baja Complejidad,Servicio de Salud Magallanes
1,126204,Hospital Naval (Puerto Williams),Mediana Complejidad,SEREMI De Magallanes y la Antártica Chilena
2,126412,Posta de Salud Rural Cameron,Baja Complejidad,Servicio de Salud Magallanes
3,126414,Posta de Salud Rural Agua Fresca,Baja Complejidad,Servicio de Salud Magallanes
4,126102,Hospital Dr. Marco Antonio Chamorro ( Porvenir),Baja Complejidad,Servicio de Salud Magallanes


In [15]:
dim_centro.to_excel(os.path.join(datos_cubos, "Dim_Centro.xlsx"), index=False)
dim_centro.to_parquet(os.path.join(datos_cubos, "Dim_Centro.parquet"), index=False)


# Cubo
Generamos una tabla multidimensional uniendo la tabla Fact con las dimensiones utilizando las llaves.

In [16]:
cubo = fact.merge(dim_especialidad,
                    how='left',
                    on = 'Especialidad_cod')

cubo = cubo.merge(dim_centro,
                  how = 'left',
                  on = 'Centro_cod')

cubo = cubo.merge(dim_periodo,
                  how ='left',
                  on = 'Mes')

cubo.sample(5)

,Centro_cod,Especialidad_cod,Mes,CNE_solicitadas,CNE_producidas,Especialidad,Especialidad_REM,Riesgo_mortalidad,Centro,Complejidad,Servicio,Trimestre,Año
1291,114105,07-011,2,25,30,COLOPROCTOLOGÍA,Si,No,Hospital Clínico Metropolitano La Florida Dra....,Alta Complejidad,Servicio de Salud Metropolitano Sur Oriente,1,2021
173,114101,07-016,8,119,152,ENFERMEDADES RESPIRATORIAS DEL ADULTO (BRONCOP...,Si,No,Complejo Hospitalario Dr. Sótero del Río (Sant...,Alta Complejidad,Servicio de Salud Metropolitano Sur Oriente,3,2021
160,114101,07-015,7,44,48,ENDOCRINOLOGÍA PEDIÁTRICA,Si,No,Complejo Hospitalario Dr. Sótero del Río (Sant...,Alta Complejidad,Servicio de Salud Metropolitano Sur Oriente,3,2021
398,114101,07-046,5,448,360,OTORRINOLARINGOLOGÍA,Si,No,Complejo Hospitalario Dr. Sótero del Río (Sant...,Alta Complejidad,Servicio de Salud Metropolitano Sur Oriente,2,2021
1694,114105,07-057,2,0,10,CIRUGÍA PLASTICA Y REPARADORA PEDIÁTRICA,Si,No,Hospital Clínico Metropolitano La Florida Dra....,Alta Complejidad,Servicio de Salud Metropolitano Sur Oriente,1,2021


In [17]:
# Exportamos a la carpeta con los cubos
cubo.to_excel(datos_cubos + 'Cubo_brechas_CNE.xlsx', index = False)

## 🏁 Discusión
1. ¿Qué campos tienen redundancias? (¿Cuál forma normal no se cumple?)
3. ¿Cuál es el propósito de introducir esta redundancia?
4. ¿Existe el riesgo que se produzan inconsistencias en los nombres de los centros o especialidades?
1. ¿Qué cambios tendría que hacer a esta tabla si tuviera:
  - Datos del 2021 al 2024?
  - Datos de más establecimientos?